In [284]:
from unstructured.partition.auto import partition
from unstructured.chunking.basic import chunk_elements
from unstructured.chunking.title import chunk_by_title
import pandas as pd
import ast
from unidecode import unidecode

In [3]:
text_pdf_filename = "samples/layout-parser-paper.pdf"
table_img_pdf_filename = "samples/embedded-images-tables.pdf"
img_pdf_filename = "samples/embedded-images.pdf"
img_filename = "samples/layout-parser-paper-fast.jpg"

prodesa_alameda_pdf_filename = "samples/prodesa/memoria-alameda-del-rio.pdf"

In [3]:
elements = partition(filename=text_pdf_filename, strategy='hi_res')
# chunks_el = chunk_elements(elements, max_characters=1024, overlap=204)

/home/parjua/.pyenv/versions/3.11.0/envs/unstructured-parser/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [4]:
print("Number of elements:", len(elements))
# print("Number of chunks:", len(chunks_el))

# max([len(chunk.text) for chunk in chunks_el])



Number of elements: 178


In [5]:
image_elememts = partition(filename=text_pdf_filename)

In [6]:
pdf_table_elements = partition(filename=table_img_pdf_filename)

In [42]:
img_pdf_elements = partition(filename=img_pdf_filename)

In [4]:
prodesa_alameda_elements = partition(filename=prodesa_alameda_pdf_filename, strategy='hi_res', pdf_infer_table_structure=True)

The pdf_infer_table_structure kwarg is deprecated. Please use skip_infer_table_types instead.
/home/parjua/.pyenv/versions/3.11.0/envs/unstructured-parser/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

In [24]:
prodesa_alameda_chunks = chunk_by_title(prodesa_alameda_elements, max_characters=15024, overlap=204)

In [12]:
print(prodesa_alameda_chunks[1].text)

% A VIVIENDA A TERRAZAS AREA TOTAL UNIDADES APTO TIPO A 75,35 4,95 80,30 180 APTO TIPO B 65,35 5,02 70,37 171 APTO TIPO C 56,57 5,02 61,59 9 APTO TIPO D 58,54 0,00 58,54 60 APTO TIPO D1 58,69 0,00 58,69 30 APTO TIPO D2 58,89 0,00 58,89 2 APTO TIPO D3 58,59 0,00 58,59 28 APTO TIPO D4 58,77 0,00 58,77 28 APTO TIPO E 49,51 0,00 49,51 2


In [25]:
len(prodesa_alameda_chunks)

7

## Parsing

In [264]:
import requests
import json

url = "http://localhost:8081"
headers = {
    "Content-Type": "application/json"
}

def get_schema(json_nodes, text, schema, outputs=["extraction", "res"], **kwargs):
    data = {
        "flow_id": "LLM_FLW_01HSEZDCTRGPSX11DDWH21K0XX",
        "outputs": outputs,
        "custom_values": {
            "language": "ES_CO",
            "identified_nodes": json_nodes,
            "optional_schema": schema,
            "contextual_text": text,
            **kwargs
        }
    }

    response = requests.post(url, headers=headers, data=json.dumps(data)).json()["result"]["user_defined_outputs"]
    print(response)

    parsed_value = response[1]["value"]
    # parsed_extraction = response[0]["value"]

    # print(parsed_extraction)
    nodes = parsed_value["entities"]
    relations = parsed_value["relations"]

    nodes_df = pd.DataFrame(nodes)
    relations_df = pd.DataFrame(relations)

    return nodes_df, relations_df

def merge_schemas(df1, df2, join_indices):
    df_merged = pd.merge(df1, df2, on=join_indices, how='outer', suffixes=('_left', '_right'))

    df_merged['props'] = df_merged.apply(
        lambda row: {**(row['props_left'] if isinstance(row['props_left'], dict) else {}),
                     **(row['props_right'] if isinstance(row['props_right'], dict) else {})},
        axis=1
    )

    df_merged['type'] = df_merged.apply(
        lambda row: row['type_left'] if not pd.isna(row['type_left']) else row['type_right'],
        axis=1
    )

    df_final = df_merged[join_indices + ['type', 'props']]

    return df_final

In [269]:
nodes_df, relations_df = None, None
count = 0

for chunk in prodesa_alameda_chunks:
    json_nodes = "None"
    if nodes_df is not None:
        json_nodes = nodes_df.to_json(orient="records")

    _nodes_df, _relations_df = get_schema(json_nodes, chunk.text, "node=Ciudad;node=Macroproyecto;node=Proyecto;node=Tipo_Apto\nedge=HAS")

    if not _nodes_df.empty:
        _nodes_df['props'] = _nodes_df['props'].apply(lambda x: {v['name']: v['value'] for v in x})

    if not _relations_df.empty:
        _relations_df['props'] = _relations_df['props'].apply(lambda x: {v['name']: v['value'] for v in x})

    if nodes_df is None:
        nodes_df = _nodes_df
    else:
        if not _nodes_df.empty:
            nodes_df = merge_schemas(nodes_df, _nodes_df, ['entity_id'])

    if relations_df is None:
        relations_df = _relations_df
    else:
        if not _relations_df.empty:
            relations_df = merge_schemas(relations_df, _relations_df, ['from_entity_id', 'from_entity_type', 'to_entity_id', 'to_entity_type'])

[{'name': 'extraction', 'description': None, 'value': '# Extracted Entities and Relationships\n\n## Entities\n\n1. **Ciudad**\n   - Nombre: Barranquilla\n   - Departamento: Atlántico\n\n2. **Macroproyecto**\n   - ID: M-R14\n   - Nombre: Alameda del Rio\n   - Ubicación: Predios de "El Volador", sobre la avenida Circunvalar, entre las carreras 38 y 46\n   - Espacio Público: 92 hectáreas\n   - Vivienda, Comercio y Equipamientos: 89 hectáreas\n   - Parques, Alamedas, Andenes y Ciclorrutas: 50 hectáreas\n   - Modelo: "Ciudad dentro de la Ciudad"\n   - Capacidad: Cerca de 22,000 familias\n\n3. **Proyecto**\n   - ID: Pardela\n   - Localización: M-R14, Alameda del Rio\n   - Estrato Socio-Económico: 3-4\n   - Área Construida Total: 61,358.83 m²\n   - Área de Vivienda: 35,817.56 m²\n   - Unidades de Vivienda: 510\n   - Torres: 4 (3 torres con 8 apartamentos por piso, 1 torre con 10 apartamentos por piso)\n   - Pisos por Torre: 15\n   - Sistema Constructivo: Muros estructurales de concreto, mampo

In [270]:
nodes_df

,entity_id,type,props
0,ALAMEDA_DEL_RIO,MACROPROYECTO,"{'id': 'M-R14', 'nombre': 'Alameda del Rio', '..."
1,BARRANQUILLA,CIUDAD,"{'nombre': 'Barranquilla', 'departamento': 'At..."
2,PARDELA,PROYECTO,"{'id': 'Pardela', 'localizacion': 'M-R14, Alam..."
3,PARQUEADERO_BICICLETAS,PARQUEADERO,"{'nombre': 'Parqueadero para Bicicletas', 'cup..."
4,PARQUEADERO_MOTOS,PARQUEADERO,"{'nombre': 'Parqueadero para Motos', 'cupos': ..."
5,PARQUEADERO_RESIDENTES,PARQUEADERO,"{'nombre': 'Parqueadero para Residentes', 'cup..."
6,PARQUEADERO_VISITANTES,PARQUEADERO,"{'nombre': 'Parqueadero para Visitantes', 'cup..."
7,TIPO_A,TIPO_APTO,"{'tipo': 'A', 'area': '80.0 m²'}"
8,TIPO_B,TIPO_APTO,"{'tipo': 'B', 'area': '70.0 m²'}"
9,TIPO_C,TIPO_APTO,"{'tipo': 'C', 'area_de_vivienda': '56.57 m²', ..."


In [271]:
relations_df

,from_entity_id,from_entity_type,to_entity_id,to_entity_type,type,props
0,ALAMEDA_DEL_RIO,MACROPROYECTO,PARDELA,PROYECTO,HAS,{}
1,BARRANQUILLA,CIUDAD,ALAMEDA_DEL_RIO,MACROPROYECTO,HAS,{}
2,PARDELA,PROYECTO,TIPO_A,TIPO_APTO,HAS,{}
3,PARDELA,PROYECTO,TIPO_B,TIPO_APTO,HAS,{}
4,PARDELA,PROYECTO,TIPO_C,TIPO_APTO,HAS,{}
5,PARDELA,PROYECTO,TIPO_D,TIPO_APTO,HAS,{}
6,PARDELA,PROYECTO,TIPO_D1,TIPO_APTO,HAS,{}
7,PARDELA,PROYECTO,TIPO_D2,TIPO_APTO,HAS,{}
8,PARDELA,PROYECTO,TIPO_D3,TIPO_APTO,HAS,{}
9,PARDELA,PROYECTO,TIPO_D4,TIPO_APTO,HAS,{}


# CSV Loading

In [274]:
def create_csv(nodes_df, relations_df, filename):
    nodes_df.to_csv(filename + "_nodes.csv", index=False)
    relations_df.to_csv(filename + "_relations.csv", index=False)

In [275]:
create_csv(nodes_df, relations_df, "outputs/prodesa_alameda")

# Load into Neo4J

In [172]:
import dotenv
import os
from neo4j import GraphDatabase

load_status = dotenv.load_dotenv("Neo4j-0c2c4bc6-Created-2024-09-16.txt")
if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [297]:
def load_csv_and_create_nodes(file_path):
    with driver.session() as session:
        df = pd.read_csv(file_path)

        for _, row in df.iterrows():
            props = ast.literal_eval(row['props'])

            filtered_props = {k: v for k, v in props.items() if v is not None}
            props_string = ", ".join([f"{unidecode(k)}: ${unidecode(k)}" for k in filtered_props.keys()])

            query = f"""
            MERGE (n:{row['type'].capitalize()} {{id: $entity_id, {props_string}}})
            """

            session.run(query, entity_id=row['entity_id'].capitalize(), **filtered_props)

def load_csv_and_create_relations(file_path):
    with driver.session() as session:
        df = pd.read_csv(file_path)

        for _, row in df.iterrows():
            props = ast.literal_eval(row['props'])

            filtered_props = {k: v for k, v in props.items() if v is not None}
            props_string = ", ".join([f"{unidecode(k)}: ${unidecode(k)}" for k in filtered_props.keys()])

            query = f"""
            MATCH (from:{row['from_entity_type'].capitalize()} {{id: $from_entity_id}})
            MATCH (to:{row['to_entity_type'].capitalize()} {{id: $to_entity_id}})
            MERGE (from)-[r:{row['type'].upper()} {{{props_string}}}]-(to)
            """

            session.run(query, from_entity_id=row['from_entity_id'].capitalize(), to_entity_id=row['to_entity_id'].capitalize(), **filtered_props)


In [298]:
load_csv_and_create_nodes("outputs/prodesa_alameda_nodes.csv")

/tmp/ipykernel_15573/144800099.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [299]:
load_csv_and_create_relations("outputs/prodesa_alameda_relations.csv")

/tmp/ipykernel_15573/144800099.py:18: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
